# Feature Selection & Feature Engineering: Predictivity by Influence Score

Modern feature selection methodologies such as step-wise backward or forward selection according to AIC/BIC presume a certain underlying model before large dimensions are penalized with an error term. This school of though is problematic because the method left data scientist, statisticians, and machine learning practitioners ensure of the source of impurity of the data set (is error coming from data or the underlying model). 

Based on this motivation, Professor Shaw-hwa Lo introduced a non-parametric feature selection technique called Influence Score (i.e. I-score). The technique is a function that takes in certain covariates $X$ and dependent variable $y$ and output a measure of how much selected $X$ influences $y$. The formula is derived from the lower bound of Bayes' accuracy. 

From previous [notebook](https://github.com/yiqiao-yin/YinsPy/blob/master/scripts/python_DS_Measure_Predictivity.ipynb), we have some working knowledge of how Influence Score is computed given selected $X$ and target $y$. This notebook I will build up on what we have done and introduce a generalized feature method. In this book, I cover

- **Review: Influence Score** I start by discussing what I covered from influence measure notebook in Data Scructures. We can produce a fixed set of partitions given a covariate matrix $X$. According to these partitions, we can compute the local average of target variable $y$ and calculate how that differs from global average of $y$, i.e. $\hat{y}$. The discrepancies between local average based on partitions and global average can be raised by the number of observations that fall in each partition. This method, Influence Score, allows us to compute how powerful given (or selected) covariate matrix $X$ is at predicting target variable $y$.

- **Backward Dropping Algorithm** Based on upon understanding, we need lots of random sampling to come up with a list of variable modules that are powerful at predicting $y$. This is the place where we design Backward Dropping Algorithm (BDA). The Backward Dropping Algorithm is a greedy searching algorithm that iteratively searching for noisy variables to drop. This system works due to a unique property of Influence Score, e.g. I-score, which is the following. Influence Score of a given collection of covariates increase if there are noisy variables in this collection and they are deleted. Influence Score of a given collection of covariates decrease if there newly added variables that do not contribute to the prediction of target variable.

- **Software Development / Product Management** Last, I land on softly coded software products.

- **Application** To test out the product, I reproduce the central idea on a brand new data set: housing price data set. Decision Tree algorithm produced about 80% on out-of-sample test set using original 18 variables. The performance seemed ok, but do we need all 18 variables? I use Influence Score and Backward Dropping Algorithm to screen for potential important variable combinations and I identified two important variables ['sqft_living', 'bedrooms']. By using only 2 variables, I am able to reproduce the 80% out-of-sample test set accuracy! 

## Influence Score

Let us recall the function we coded from previous notebook in the following

In [1]:
# Define function
def iscore(X, y):
    # Environment Initiation
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    import random
    
    # Create Partition
    partition = X.iloc[:, 0].astype(str)
    if X.shape[1] >= 2:
        for i in range(X.shape[1]-1):
            partition = partition.astype(str) + '_' + X.iloc[:, i].astype(str)
    else:
        partition = partition

    # Local Information
    list_of_partitions = pd.DataFrame(partition.value_counts())
    Pi = pd.DataFrame(list_of_partitions.index)
    local_n = pd.DataFrame(list_of_partitions.iloc[:, :])

    # Compute Influence Score:
    import collections
    list_local_mean = []
    Y_bar = y.mean()
    local_mean_vector = []
    grouped = pd.DataFrame({'y': y, 'X': partition})
    local_mean_vector = pd.DataFrame(grouped.groupby('X').mean())
    iscore = np.mean(np.array(local_n).reshape(1, local_n.shape[0]) * np.array((local_mean_vector['y'] - Y_bar)**2))/np.std(y)
    
    # Output
    return {
        'X': X,
        'y': y,
        'Local Mean Vector': local_mean_vector,
        'Global Mean': Y_bar,
        'Partition': Pi,
        'Number of Samples in Partition': local_n,
        'Influence Score': iscore}
# End of function

Let us try it on a real data set.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [3]:
# Data
data = pd.read_csv('~/OneDrive/Documents/YinsPy/data/startups_invest.csv')
X = data.iloc[:, :-1]
y = data.iloc[:, data.shape[1] - 1]
y = (y > np.mean(y)).astype(int)
State = pd.get_dummies(X.iloc[:, 3], drop_first=True)
X = pd.concat([X.iloc[:, :3], State], axis=1)
print(X.head(2))

   R&D Spend  Administration  Marketing Spend  Florida  New York
0   165349.2       136897.80        471784.10        0         1
1   162597.7       151377.59        443898.53        0         0


In [4]:
newX = pd.DataFrame([])
for j in range(X.shape[1]):
    feature = X.iloc[:, j]
    feature = (feature > feature.mean()).astype(int)
    newX = pd.concat([newX, feature], axis=1)
print(newX.head(2))
print(newX.shape)

   R&D Spend  Administration  Marketing Spend  Florida  New York
0          1               1                1        0         1
1          1               1                1        0         0
(50, 5)


In [5]:
# Random Sampling
# Note: Python executes each code box independently. Once this box is executed
#       you have to start from previous code box to recover the original 
#       covariate matrix first. If this is not done, the covariate matrix 
#       *newX* will get smaller and smaller.
num_initial_draw = 3
newX = newX.iloc[:, random.sample(range(newX.shape[1]), num_initial_draw)]
print(newX.head(3))

   R&D Spend  Marketing Spend  Administration
0          1                1               1
1          1                1               1
2          1                1               0


In [6]:
# Try
testresult = iscore(X=newX, y=y)
print(testresult['Partition'])
print(testresult['X'].head(3))
print(testresult['Influence Score'])

       0
0  1_1_1
1  0_0_0
2  0_0_1
3  1_1_0
   R&D Spend  Marketing Spend  Administration
0          1                1               1
1          1                1               1
2          1                1               0
4.241835717334233


## The Backward Dropping Algorithm

Let us introduce a greedy backward selection algorithm based on the unique property of Influence Score (i.e. I-score). If selected covariate $X$ carries crucial information about dependent variable $y$, we expect to observe a high value for Influence Score (i.e. I-score). If somehow selected covariate $X$ carries noisy variables that damage the purity of $X$ to predict $y$, we expect this measure to decrease. In addition, the more noisy variable selected covariate matrix carries, the lower the Influence Score (e.g. I-score). Hence, due to this property, we develop the Backward Dropping Algorithm.

In [7]:
newX = pd.DataFrame([])
for j in range(X.shape[1]):
    feature = X.iloc[:, j]
    feature = (feature > feature.mean()).astype(int)
    newX = pd.concat([newX, feature], axis=1)
print(newX.head(2))
print(newX.shape)

# Random Sampling
# Note: Python executes each code box independently. Once this box is executed
#       you have to start from previous code box to recover the original 
#       covariate matrix first. If this is not done, the covariate matrix 
#       *newX* will get smaller and smaller.
num_initial_draw = 4
newX = newX.iloc[:, random.sample(range(newX.shape[1]), num_initial_draw)]
print(newX.head(3))

   R&D Spend  Administration  Marketing Spend  Florida  New York
0          1               1                1        0         1
1          1               1                1        0         0
(50, 5)
   Florida  R&D Spend  Marketing Spend  Administration
0        0          1                1               1
1        0          1                1               1
2        1          1                1               0


In [8]:
# Compute Influence Score, e.g. I-score
testresult = iscore(X=newX, y=y)
print(testresult['Influence Score'])

2.085093762462482


In [9]:
newX_copy = newX
iscorePath = []
selectedX = {}
for j in range(newX_copy.shape[1]-1):
    unit_scores = []
    for i in range(newX.shape[1]):
        unit_scores.append(iscore(X=newX.iloc[:, :].drop([str(newX.columns[i])], axis=1), y=y)['Influence Score'])
        #print(i, unit_scores, np.max(unit_scores), unit_scores.index(max(unit_scores)))
    iscorePath.append(np.max(unit_scores))
    to_drop = unit_scores.index(max(unit_scores))
    newX = newX.iloc[:, :].drop([str(newX.columns[to_drop])], axis=1).head(3)
    selectedX[str(j)] = newX

In [10]:
print(iscorePath)
print(selectedX[str(iscorePath.index(max(iscorePath)))])

[4.544159627264301, 2.0447423868476524, 2.0447423868476524]
   Florida  R&D Spend  Administration
0        0          1               1
1        0          1               1
2        1          1               0


## Software Development / Product Managmeent

### Soft Code

Let us soft code the procedure of Backward Dropping Algorithm.

In [11]:
# Define function
def FSFE_BDA(newX, y, num_initial_draw = 4):
    # Random Sampling
    newX = newX.iloc[:, random.sample(range(newX.shape[1]), num_initial_draw)]
    
    # BDA
    newX_copy = newX
    iscorePath = []
    selectedX = {}
    for j in range(newX_copy.shape[1]-1):
        unit_scores = []
        for i in range(newX.shape[1]):
            unit_scores.append(iscore(X=newX.iloc[:, :].drop([str(newX.columns[i])], axis=1), y=y)['Influence Score'])
            #print(i, unit_scores, np.max(unit_scores), unit_scores.index(max(unit_scores)))
        iscorePath.append(np.max(unit_scores))
        to_drop = unit_scores.index(max(unit_scores))
        newX = newX.iloc[:, :].drop([str(newX.columns[to_drop])], axis=1).head(3)
        selectedX[str(j)] = newX
        
    # Final Output
    finalX = pd.DataFrame(selectedX[str(iscorePath.index(max(iscorePath)))])

    # Output
    return {
        'Path': iscorePath,
        'MaxIscore': np.max(iscorePath),
        'newX': finalX
    }
# End of function

Let us try it!

In [12]:
# Data
data = pd.read_csv('~/OneDrive/Documents/YinsPy/data/startups_invest.csv')
X = data.iloc[:, :-1]
y = data.iloc[:, data.shape[1] - 1]
y = (y > np.mean(y)).astype(int)
State = pd.get_dummies(X.iloc[:, 3], drop_first=True)
X = pd.concat([X.iloc[:, :3], State], axis=1)
print(X.head(2))

newX = pd.DataFrame([])
for j in range(X.shape[1]):
    feature = X.iloc[:, j]
    feature = (feature > feature.mean()).astype(int)
    newX = pd.concat([newX, feature], axis=1)
print(newX.head(2))
print(newX.shape)

   R&D Spend  Administration  Marketing Spend  Florida  New York
0   165349.2       136897.80        471784.10        0         1
1   162597.7       151377.59        443898.53        0         0
   R&D Spend  Administration  Marketing Spend  Florida  New York
0          1               1                1        0         1
1          1               1                1        0         0
(50, 5)


In [13]:
testBDA = FSFE_BDA(newX, y, num_initial_draw = 4)

In [14]:
testBDA['Path']

[4.633695837019997, 2.0447423868476524, 2.0447423868476524]

In [15]:
testBDA['MaxIscore']

4.633695837019997

In [16]:
testBDA['newX'].head(3)

,R&D Spend,Administration,New York
0,1,1,1
1,1,1,0
2,1,0,0


In [17]:
class InteractionBasedLearning:

    # Define function
    def iscore(X, y):
        # Environment Initiation
        import numpy as np
        import matplotlib.pyplot as plt
        import pandas as pd
        import random

        # Create Partition
        partition = X.iloc[:, 0].astype(str)
        if X.shape[1] >= 2:
            for i in range(X.shape[1]-1):
                partition = partition.astype(str) + '_' + X.iloc[:, i].astype(str)
        else:
            partition = partition

        # Local Information
        list_of_partitions = pd.DataFrame(partition.value_counts())
        Pi = pd.DataFrame(list_of_partitions.index)
        local_n = pd.DataFrame(list_of_partitions.iloc[:, :])

        # Compute Influence Score:
        import collections
        list_local_mean = []
        Y_bar = y.mean()
        local_mean_vector = []
        grouped = pd.DataFrame({'y': y, 'X': partition})
        local_mean_vector = pd.DataFrame(grouped.groupby('X').mean())
        iscore = np.mean(np.array(local_n).reshape(1, local_n.shape[0]) * np.array((local_mean_vector['y'] - Y_bar)**2))/np.std(y)

        # Output
        return {
            'X': X,
            'y': y,
            'Local Mean Vector': local_mean_vector,
            'Global Mean': Y_bar,
            'Partition': Pi,
            'Number of Samples in Partition': local_n,
            'Influence Score': iscore}
    # End of function
    
    # Define function
    def FSFE_BDA(newX, y, num_initial_draw = 4):
        # Random Sampling
        newX = newX.iloc[:, random.sample(range(newX.shape[1]), num_initial_draw)]

        # BDA
        newX_copy = newX
        iscorePath = []
        selectedX = {}
        for j in range(newX_copy.shape[1]-1):
            unit_scores = []
            for i in range(newX.shape[1]):
                unit_scores.append(InteractionBasedLearning.iscore(X=newX.iloc[:, :].drop([str(newX.columns[i])], axis=1), y=y)['Influence Score'])
                #print(i, unit_scores, np.max(unit_scores), unit_scores.index(max(unit_scores)))
            iscorePath.append(np.max(unit_scores))
            to_drop = unit_scores.index(max(unit_scores))
            newX = newX.iloc[:, :].drop([str(newX.columns[to_drop])], axis=1).head(3)
            selectedX[str(j)] = newX

        # Final Output
        finalX = pd.DataFrame(selectedX[str(iscorePath.index(max(iscorePath)))])

        # Output
        return {
            'Path': iscorePath,
            'MaxIscore': np.max(iscorePath),
            'newX': finalX
        }
    # End of function

In [18]:
# Data
data = pd.read_csv('~/OneDrive/Documents/YinsPy/data/startups_invest.csv')
X = data.iloc[:, :-1]
y = data.iloc[:, data.shape[1] - 1]
y = (y > np.mean(y)).astype(int)
State = pd.get_dummies(X.iloc[:, 3], drop_first=True)
X = pd.concat([X.iloc[:, :3], State], axis=1)
print(X.head(2))

newX = pd.DataFrame([])
for j in range(X.shape[1]):
    feature = X.iloc[:, j]
    feature = (feature > feature.mean()).astype(int)
    newX = pd.concat([newX, feature], axis=1)
print(newX.head(2))
print(newX.shape)

   R&D Spend  Administration  Marketing Spend  Florida  New York
0   165349.2       136897.80        471784.10        0         1
1   162597.7       151377.59        443898.53        0         0
   R&D Spend  Administration  Marketing Spend  Florida  New York
0          1               1                1        0         1
1          1               1                1        0         0
(50, 5)


In [19]:
InteractionBasedLearning.FSFE_BDA

<function __main__.InteractionBasedLearning.FSFE_BDA(newX, y, num_initial_draw=4)>

In [20]:
testResult = InteractionBasedLearning.FSFE_BDA(newX=newX, y=y, num_initial_draw=5)

In [21]:
testResult['MaxIscore']

3.2195014827924004

In [22]:
testResult['newX']

,Florida,R&D Spend,New York,Marketing Spend
0,0,1,1,1
1,0,1,0,1
2,1,1,0,1


## Load .py Script Class Object

We can also save the above *class* object in a *.py* script so that in the future we can load this script.

In [23]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [24]:
# Get Data
house_sales = pd.read_csv('../data/kc_house_data.csv')
house_sales.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


In [25]:
print(house_sales.shape)

(21613, 21)


In [26]:
# Clean Data
data = house_sales
X = data.iloc[:, :].drop(['id', 'date', 'price'], axis=1)
y = (data['price'] > np.mean(data['price'])).astype(int)
print(X.head(2))
print(y)

   bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0         3       1.00         1180      5650     1.0           0     0   
1         3       2.25         2570      7242     2.0           0     0   

   condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0          3      7        1180              0      1955             0   
1          3      7        2170            400      1951          1991   

   zipcode      lat     long  sqft_living15  sqft_lot15  
0    98178  47.5112 -122.257           1340        5650  
1    98125  47.7210 -122.319           1690        7639  
0        0
1        0
2        0
3        1
4        0
        ..
21608    0
21609    0
21610    0
21611    0
21612    0
Name: price, Length: 21613, dtype: int32


In [27]:
newX = pd.DataFrame([])
for j in range(X.shape[1]):
    feature = X.iloc[:, j]
    feature = (feature > feature.mean()).astype(int)
    newX = pd.concat([newX, feature], axis=1)
print(newX.head(2))
print(newX.shape)

   bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0         0          0            0         0       0           0     0   
1         0          1            1         0       1           0     0   

   condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0          0      0           0              0         0             0   
1          0      0           1              1         0             1   

   zipcode  lat  long  sqft_living15  sqft_lot15  
0        1    0     0              0           0  
1        1    1     0              0           0  
(21613, 18)


In [41]:
%run "../scripts/InteractionBasedLearning.py"

In [42]:
InteractionBasedLearning.FSFE_BDA

<function __main__.InteractionBasedLearning.FSFE_BDA(newX, y, num_initial_draw=4)>

In [45]:
oneDraw = InteractionBasedLearning.FSFE_BDA(newX=newX, y=y, num_initial_draw=7)

In [46]:
print(np.array(oneDraw['newX'].columns), oneDraw['MaxIscore'])

['sqft_living' 'long'] 1227.557327644066


In [51]:
import time
start = time.time()
listVariableModule = []
listInfluenceScore = []
for i in range(5):
    oneDraw = InteractionBasedLearning.FSFE_BDA(newX=newX, y=y, num_initial_draw=7)
    listVariableModule.append([np.array(oneDraw['newX'].columns)])
    listInfluenceScore.append(oneDraw['MaxIscore'])
    print(f'Finished round {i}')
end = time.time()
print(end - start)

Finished round 0
Finished round 1
Finished round 2
Finished round 3
Finished round 4
30.92867612838745


In [52]:
listVariableModule

[[array(['lat', 'waterfront', 'sqft_basement'], dtype=object)],
 [array(['sqft_living', 'bedrooms'], dtype=object)],
 [array(['grade'], dtype=object)],
 [array(['sqft_above'], dtype=object)],
 [array(['bathrooms'], dtype=object)]]

In [53]:
listInfluenceScore

[1215.6598409240405,
 1227.557327644066,
 1223.2110414689878,
 827.1103194007787,
 558.2045022216643]

The above script did the following work and this is how we interpret the results:

- The script runs functions loaded from *.py* script and the functions are executed five times. There is a time check in the end that is about 30 seconds.

- The function outputs an array of variable modules and an array of influence scores (order matters and the are one-one map). For examplt, the first array from both lists are ['lat', 'waterfront', 'sqft_basement'] and the influence measure is about 1216. This is much higher than ['sqft_above'] and ['bathrooms'] but not as high as ['sqft_living', 'bedrooms'] which has an influence measure of 1228.

- We can conclude that to build an explanable machine learning algorithm to predict whether housing price for a particular place is higher than average with the variable module ['sqft_living', 'bedrooms'] it is reasonable to use selected variable modules instead of the entire data set.

In [156]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(newX, y, test_size = 0.3, random_state = 0)

In [157]:
%run "../scripts/YinsML.py"

In [158]:
testResult = YinsML.DecisionTree_Classifier(X_train, X_test, y_train, y_test)
print(testResult['Data']['X_train'].head(3))
print(testResult['Test Result']['confusion_test'])
print(testResult['Test Result']['test_acc'])

       bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
1468          1          0            0         0       0           0     0   
15590         0          0            0         0       0           0     0   
18552         1          1            1         0       1           0     0   

       condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
1468           0      0           0              0         0             0   
15590          0      0           0              0         0             0   
18552          0      1           1              0         1             0   

       zipcode  lat  long  sqft_living15  sqft_lot15  
1468         1    1     0              0           0  
15590        1    1     0              0           0  
18552        0    1     1              1           0  
      0     1
0  3885  1001
1   249  1349
0.8072177668106107


Let us select import variables and then repeat Decision Tree algorithm.

In [159]:
listVariableModule

[[array(['lat', 'waterfront', 'sqft_basement'], dtype=object)],
 [array(['sqft_living', 'bedrooms'], dtype=object)],
 [array(['grade'], dtype=object)],
 [array(['sqft_above'], dtype=object)],
 [array(['bathrooms'], dtype=object)]]

In [160]:
informativeX = pd.DataFrame(pd.concat([newX['sqft_living'], newX['bedrooms']], axis=1))
informativeX.head(3)

,sqft_living,bedrooms
0,0,0
1,1,0
2,0,0


In [161]:
testResult = YinsML.DecisionTree_Classifier(X_train, X_test, y_train, y_test)
print(testResult['Data']['X_train'].head(3))
print(testResult['Test Result']['confusion_test'])
print(testResult['Test Result']['test_acc'])

       bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
1468          1          0            0         0       0           0     0   
15590         0          0            0         0       0           0     0   
18552         1          1            1         0       1           0     0   

       condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
1468           0      0           0              0         0             0   
15590          0      0           0              0         0             0   
18552          0      1           1              0         1             0   

       zipcode  lat  long  sqft_living15  sqft_lot15  
1468         1    1     0              0           0  
15590        1    1     0              0           0  
18552        0    1     1              1           0  
      0     1
0  3885  1001
1   249  1349
0.8072177668106107


We observe that accuracy on test set (out-of-sample) on original data set is about 81%. Now that we shrink dimension to only 2 variables and the performance is still 81%. This means that it is very possible the rest of the variables do not generate real values in predicting the target variable.